In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import sklearn as skl
import sys

In [4]:
np.set_printoptions(threshold=sys.maxsize)

In [5]:
path = "./data/"

In [6]:
filename = "./a.txt"

In [26]:
with open(path+filename, "r") as f:
    lines = f.readlines()
D, I, S, V, F = list(map(int, lines[0].strip().split(" ")))

In [48]:
streets = pd.DataFrame()
for s in range(1,S+1):
    data = {}
    elems = list(map(str, lines[s].strip().split(" ")))
    data['B'] = int(elems[0])
    data['E'] = int(elems[1])
    data['street_name'] = elems[2]
    data['L'] = int(elems[3])
    streets = streets.append(data, ignore_index=True)

In [49]:
streets

,B,E,L,street_name
0,2.0,0.0,1.0,rue-de-londres
1,0.0,1.0,1.0,rue-d-amsterdam
2,3.0,1.0,1.0,rue-d-athenes
3,2.0,3.0,2.0,rue-de-rome
4,1.0,2.0,3.0,rue-de-moscou


In [50]:
cars = pd.DataFrame()
for v in range(S+1,V+S+1):
    data = {}
    elems = list(map(str, lines[v].strip().split(" ")))
    data['P'] = int(elems[0])
    for p in range(data['P']):
        data['street_'+str(p+1)] = elems[p+1]
    cars = cars.append(data, ignore_index=True)

In [51]:
cars

,P,street_1,street_2,street_3,street_4
0,4.0,rue-de-londres,rue-d-amsterdam,rue-de-moscou,rue-de-rome
1,3.0,rue-d-athenes,rue-de-moscou,rue-de-londres,NaN


In [ ]:
load_dict = load_df("b_little_bit_of_everything.in")

In [ ]:
for FILE  in os.listdir("."):
    if not FILE.endswith(".in"):
        continue
        
    print(f"PROCESANDO FICHERO -> {FILE}")
        
    load_dict=load_df(FILE)
    s = solver(load_dict)
    output_funct(s, f"{FILE}.out")